In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np

In [14]:
# Load sensor data
sensor_data = pd.read_csv('../static/data/slc_daily_pm2.5_pm10_2016to2024.csv')

# Assuming your sensor data has columns 'latitude' and 'longitude'

# Convert sensor data DataFrame to GeoDataFrame
sensor_data['geometry'] = sensor_data.apply(lambda row: Point(row.longitude, row.latitude), axis=1)
sensor_gdf = gpd.GeoDataFrame(sensor_data, geometry='geometry')

# Load GeoJSON file
geojson_gdf = gpd.read_file('../static/data/converted_geojson_data.geojson')

In [15]:
# Ensure both GeoDataFrames use the same CRS
sensor_gdf = sensor_gdf.set_crs(geojson_gdf.crs)

# Perform spatial join
joined_gdf = gpd.sjoin(sensor_gdf, geojson_gdf, how="left", op='within')


/Users/juliakauffmann/miniforge3/envs/capstonev/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [16]:
# Function to categorize based on 'Lowmod_pct'
def categorize(lowmod_pct):
    if lowmod_pct > 0.75:
        return 'red'
    elif lowmod_pct > 0.5:
        return 'orange'
    elif lowmod_pct > 0.25:
        return 'green'
    else:
        return 'blue'

# Apply categorization
joined_gdf['category'] = joined_gdf['Lowmod_pct'].apply(categorize)

# Log the categories (or export to CSV)
print(joined_gdf[['sensor_id', 'category','Countyname']])



        sensor_id category        Countyname
0              77      red  Salt Lake County
1              77      red  Salt Lake County
2              77      red  Salt Lake County
3              77      red  Salt Lake County
4              77      red  Salt Lake County
...           ...      ...               ...
282304     208783     blue      Weber County
282305     208783     blue      Weber County
282306     208783     blue      Weber County
282307     208781    green      Weber County
282308     208781    green      Weber County

[282309 rows x 3 columns]


In [19]:
df.rename(columns = {'Countyname':'county'},inplace=True)

In [20]:
df = joined_gdf.groupby('sensor_id').first().reset_index()[['sensor_id', 'category','Countyname']]
df.rename(columns = {'Countyname':'county'},inplace=True)
#export to CSV
df.to_csv('../static/data/sensor_categories.csv', index=False)

In [21]:
df

,sensor_id,category,county
0,77,red,Salt Lake County
1,443,red,Weber County
2,525,green,Salt Lake County
3,984,blue,Salt Lake County
4,992,green,Salt Lake County
...,...,...,...
217,204009,red,Salt Lake County
218,204315,green,Salt Lake County
219,207743,blue,Salt Lake County
220,208781,green,Weber County
